In [3]:
from trip_kinematics.HomogenTransformationMatrix import TransformationMatrix
from trip_kinematics.KinematicGroup import KinematicGroup, Transformation
import sympy


In [6]:
A = Transformation(name='gimbal_joint',values={'rx': 0, 'ry': 0, 'rz': 0}, state_variables=['rx', 'ry', 'rz'])
print(A.state)
print(A.get_convention())

{'rx': 0, 'ry': 0, 'rz': 0}


TypeError: get_convention() missing 1 required positional argument: 'state'

### Idee erlaube transformation selbst symbolische representation zu returnen!

In [ ]:
def get_symbolic_rep(transformation):
    return 0